In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('fifa_spotify_data_cleaned.csv')

In [3]:
df['album_release_date'] = df['album_release_date'].astype('datetime64')

In [4]:
df = df.fillna('None')

In [5]:
# Create playlist year column
df['playlist_year'] = df.playlist_name.apply(lambda x: int(x[5:7]))

In [6]:
df = df.sort_values(by='playlist_year')

In [7]:
df

,playlist_name,num_tracks,track_name,track_duration,track_explicit,track_popularity,acousticness,danceability,energy,instrumentalness,...,pop,indie,rock,edm,hip hop,dance,avg_artist_following,avg_artist_popularity,single,playlist_year
104,FIFA 14 Soundtrack,36,Boa Noite,166278,0,12,0.198000,0.508,0.800,0.000000,...,1,1,0,0,1,0,480668.0,55.000000,1,14
96,FIFA 14 Soundtrack,36,Love Me Again,239894,0,0,0.003930,0.497,0.888,0.000562,...,1,0,0,0,0,0,603997.0,71.000000,1,14
95,FIFA 14 Soundtrack,36,Don't Forget Who You Are,204946,0,39,0.000732,0.462,0.953,0.000010,...,1,1,1,0,0,0,204044.0,63.000000,1,14
94,FIFA 14 Soundtrack,36,Get Down,187591,1,0,0.005010,0.736,0.949,0.000000,...,0,0,0,0,0,0,2638.0,9.000000,1,14
93,FIFA 14 Soundtrack,36,On Our Way,193853,0,55,0.008400,0.581,0.793,0.000000,...,1,1,1,0,0,0,102489.0,49.000000,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,FIFA 20 Soundtrack,41,Positive,337800,0,46,0.010700,0.608,0.654,0.074500,...,1,1,1,1,0,1,538350.0,63.000000,0,20
23,FIFA 20 Soundtrack,41,Zulu Screams (feat. Maleek Berry & Bibi Bourelly),177706,0,45,0.172000,0.751,0.937,0.014300,...,0,0,0,0,1,0,639805.0,73.000000,0,20
22,FIFA 20 Soundtrack,41,Go Wild,197681,0,52,0.077700,0.677,0.866,0.000088,...,0,1,0,0,0,0,3545.0,40.000000,1,20
31,FIFA 20 Soundtrack,41,Where Do I Begin,119545,0,41,0.014600,0.757,0.695,0.000000,...,0,1,0,0,1,0,49518.0,55.000000,0,20


In [8]:
df.columns

Index(['playlist_name', 'num_tracks', 'track_name', 'track_duration',
       'track_explicit', 'track_popularity', 'acousticness', 'danceability',
       'energy', 'instrumentalness', 'key', 'loudness', 'mode', 'tempo',
       'valence', 'artist1_name', 'artist2_name', 'artist3_name', 'album_name',
       'album_type', 'album_popularity', 'album_release_date', 'genres', 'pop',
       'indie', 'rock', 'edm', 'hip hop', 'dance', 'avg_artist_following',
       'avg_artist_popularity', 'single', 'playlist_year'],
      dtype='object')

# App

In [9]:
app = dash.Dash(__name__)
server = app.server

In [10]:
app.layout = html.Div([
    
    html.H1('FIFA Playlist Data Visualization', style = {'text-align': 'center'}), 
    html.Br(),
    
    html.H2('Audio Features', style = {'text-align': 'center'}),
    
    dcc.Dropdown(id="selected_audio_feature",
                 options=[
                     {'label': 'Acousticness', 'value': 'acousticness'}, 
                     {'label': 'Danceability', 'value': 'danceability'},
                     {'label': 'Energy', 'value': 'energy'},
                     {'label': 'Instrumentalness', 'value': 'instrumentalness'},
                     {'label': 'Key', 'value': 'key'},
                     {'label': 'Loudness', 'value': 'loudness'},
                     {'label': 'Mode', 'value': 'mode'},
                     {'label': 'Tempo', 'value': 'tempo'},
                     {'label': 'Valence', 'value': 'valence'}],
                 multi=False,
                 value='acousticness',
                 style={'width': "75%", 'diplay': 'inline-block'}
                 ),

    dcc.Graph(id='aud_vs_year_lineplot', figure={}),
    html.Br(),
    
    dcc.Dropdown(id="selected_year",
                 options=[
                     {'label': 'FIFA 2020', 'value': 20}, 
                     {'label': 'FIFA 2019', 'value': 19},
                     {'label': 'FIFA 2018', 'value': 18},
                     {'label': 'FIFA 2017', 'value': 17},
                     {'label': 'FIFA 2016', 'value': 16},
                     {'label': 'FIFA 2015', 'value': 15},
                     {'label': 'FIFA 2014', 'value': 14}],
                 multi=False,
                 value=20,
                 style={'width': "75%", 'diplay': 'inline-block'}
                 ),
    
    dcc.Graph(id='aud_dist_boxplot', figure={}),
    html.Br(),
    
    dcc.Graph(id='track_artist_popularity', figure= px.scatter(
                                            data_frame = df,
                                            x = df.avg_artist_popularity,
                                            y = df.track_popularity,
                                            color = df.playlist_name,
                                            title = 'Track vs. Artist Popularity',
                                            hover_data = ['track_name','artist1_name', 'artist2_name', 'artist3_name']
                                        )),
    html.Br(),
    
    dcc.Graph(id='track_album_popularity', figure= px.scatter(
                                            data_frame = df[df.single == 0],
                                            x = df[df.single == 0].album_popularity,
                                            y = df[df.single == 0].track_popularity,
                                            color = df[df.single == 0].playlist_name,
                                            title = 'Track vs. Album Popularity',
                                            hover_data = ['track_name','album_name']
                                        ))
    

])

In [11]:
@app.callback(
    Output(component_id='aud_vs_year_lineplot', component_property='figure'),
    [Input(component_id='selected_audio_feature', component_property='value')]
)

def update_graph(selected_audio_feature):
    
    filtered_df = df.copy()
    filtered_df = filtered_df.groupby(['playlist_year']).mean()
    
    fig = px.line(
        data_frame = filtered_df,
        x = filtered_df.index,
        y = filtered_df[selected_audio_feature],
        title = 'Audio Feature vs. Year'
    )

    return fig

In [12]:
@app.callback(
    Output(component_id='aud_dist_boxplot', component_property='figure'),
    [Input(component_id='selected_year', component_property='value')]
)

def update_graph(selected_year):
    
    filtered_df = df.copy()
    filtered_df = filtered_df[filtered_df.playlist_year == selected_year]

    fig = make_subplots(rows=1, cols=9, subplot_titles=("Acousticness", "Danceability", "Energy", "Instrumentalness", 
                                                        "Key", "Loudness", "Mode", "Tempo", "Valence"))

    fig.add_trace(
        go.Box(y=filtered_df.acousticness, boxmean=True),
        row=1, col=1
    )
    fig.add_trace(
        go.Box(y=filtered_df.danceability, boxmean=True),
        row=1, col=2
    )
    fig.add_trace(
        go.Box(y=filtered_df.energy, boxmean=True),
        row=1, col=3
    )
    fig.add_trace(
        go.Box(y=filtered_df.instrumentalness, boxmean=True),
        row=1, col=4
    )
    fig.add_trace(
        go.Box(y=filtered_df.key, boxmean=True),
        row=1, col=5
    )
    fig.add_trace(
        go.Box(y=filtered_df.loudness, boxmean=True),
        row=1, col=6
    )
    fig.add_trace(
        go.Box(y=filtered_df['mode'], boxmean=True),
        row=1, col=7
    )
    fig.add_trace(
        go.Box(y=filtered_df.tempo, boxmean=True),
        row=1, col=8
    )
    fig.add_trace(
        go.Box(y=filtered_df.valence, boxmean=True),
        row=1, col=9
    )

    fig.update_layout(height=400, width=1500,
                      title_text="Distribution of Audio Features in the FIFA 20{} Playlist".format(selected_year))

    return fig

In [13]:
if __name__=='__main__':
    app.run_server(debug=False, port=8080, host='0.0.0.0')

Dash is running on http://0.0.0.0:8080/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jul/2020 08:09:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 08:09:25] "GET /_dash-component-suites/dash_renderer/react@16.v1_5_1m1595294030.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 08:09:25] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_5_1m1595294030.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 08:09:25] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_10_1m1595294031.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 08:09:25] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_5_1m1595294030.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 08:09:25] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_5_1m1595294030.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 08:09:25] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_10_1m1595294032.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 08: